<a href="https://colab.research.google.com/github/elenaslunap/64WordleSolver/blob/main/src/proyecto_chatbots.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Wordle Solver
## Proyecto Final para el curso de Chatbots e Inteligencia Artificial

Elaborado por:
- Elena Sofía Luna Palacio
- Guillermo Arredondo Renero


In [ ]:
# !pip install requests
# !pip install tqdm
# !pip install transformers
# !pip install datasets
# !pip install torch

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 12.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 9.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 11.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 16.5 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [ ]:
# Importamos las bibliotecas necesarias
import torch
import requests
from io import BytesIO

%matplotlib inline
import matplotlib.pyplot as plt
from tqdm import tqdm

from transformers import pipeline

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [ ]:
state = {"correct": {}, "present": {}, "absent": set()}
for i, (g, f) in enumerate(zip("hola", ["G", "B", "B", "Y"])):
  print(i, g, f)
  if f == "G":  # Green
    state["correct"][i] = g
  elif f == "Y":  # Yellow
    state["present"][i] = g
  elif f == "B":  # Black
    state["absent"].add(g)

print(state)

0 h G
1 o B
2 l B
3 a Y
{'correct': {0: 'h'}, 'present': {'a'}, 'absent': {'l', 'o'}}


In [ ]:
def parse_feedback(guess, feedback):
    state = {"correct": {}, "present": set(), "absent": set()}
    for i, (g, f) in enumerate(zip(guess, feedback)):
        if f == "G":  # Green
            state["correct"][i] = g
        elif f == "Y":  # Yellow
            state["present"].add(g)
        elif f == "B":  # Black
            state["absent"].add(g)
    return state

In [ ]:
generator = pipeline("text-generation", model="gpt-2")

def generate_candidates(partial_word, constraints):
    # Use constraints (e.g., excluded letters, included positions) to filter suggestions
    suggestions = generator(partial_word, max_length=10, num_return_sequences=5)
    return [s["generated_text"] for s in suggestions]

def generate_candidates(constraints):






In [ ]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer

# Cargar el tokenizador y el modelo
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
model = GPT2LMHeadModel.from_pretrained("gpt2")

In [ ]:
board_states = [{"correct": {}, "present": set(), "absent": set()} for _ in range(64)]

In [ ]:
import torch
from transformers import GPT2Tokenizer, GPT2Model, Trainer, TrainingArguments, TextDataset, DataCollatorForLanguageModeling

# Load the dataset from a text file
def load_dataset(file_path, tokenizer, block_size=128):
    return TextDataset(
        tokenizer=tokenizer,
        file_path=file_path,
        block_size=block_size
    )

# Fine-tune GPT-2 on the Wordle words
def fine_tune_gpt2(train_file):
    # Load the GPT-2 tokenizer and model
    model_name = "gpt2"  # Base GPT-2
    tokenizer = GPT2Tokenizer.from_pretrained(model_name)
    model = GPT2Model.from_pretrained(model_name)

    # Add padding token if missing
    if tokenizer.pad_token is None:
        tokenizer.pad_token = tokenizer.eos_token

    # Load the dataset
    dataset = load_dataset(train_file, tokenizer)

    # Define the data collator for language modeling
    data_collator = DataCollatorForLanguageModeling(
        tokenizer=tokenizer,
        mlm=False  # Set to False for causal language modeling
    )

    # Training arguments
    training_args = TrainingArguments(
        output_dir="./gpt2_wordle_finetuned",
        overwrite_output_dir=True,
        num_train_epochs=3,  # Fine-tuning for 3 epochs
        per_device_train_batch_size=8,
        save_steps=500,
        save_total_limit=2,
        logging_dir="./logs",
        logging_steps=100,
    )

    # Initialize Trainer
    trainer = Trainer(
        model=model,
        args=training_args,
        data_collator=data_collator,
        train_dataset=dataset,
    )

    # Train the model
    print("Starting Fine-tuning...")
    trainer.train()

    # Save the fine-tuned model
    print("Saving fine-tuned model...")
    trainer.save_model("./gpt2_wordle_finetuned")
    tokenizer.save_pretrained("./gpt2_wordle_finetuned")

    print("Fine-tuning complete!")

# File path to your Wordle words dataset
train_file = "wordle-answers-alphabetical.txt"

# Run the fine-tuning function
fine_tune_gpt2(train_file)


/usr/local/lib/python3.10/dist-packages/transformers/data/datasets/language_modeling.py:53: FutureWarning: This dataset will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/main/examples/pytorch/language-modeling/run_mlm.py
  warnings.warn(


Starting Fine-tuning...


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


TypeError: GPT2Model.forward() got an unexpected keyword argument 'labels'

Generar conjutnos de 64 palabras para imitar